In [0]:
from google.colab import drive 
drive.mount('/content/gdrive/')



Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
!ls gdrive/


'My Drive'


In [0]:
!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json




ls: cannot access 'kaggle.json': No such file or directory
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 2.1MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=46906364b79dc23aa6acafef3b0725bd67b60df06bb98b00904f292e5ee9cb35
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [0]:
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16

     |████████████████████████████████| 25.0MB 1.3MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


     |████████████████████████████████| 30.6MB 1.4MB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [0]:
!pip install kmc2



     |████████████████████████████████| 102kB 2.4MB/s 
     |████████████████████████████████| 163kB 8.4MB/s 
  Created wheel for kmc2: filename=kmc2-0.1-cp36-cp36m-linux_x86_64.whl size=252222 sha256=874914e909fb126fc1c79f7a123fd1420b2eb61cc0387f214804574b4794a3db
  Stored in directory: /root/.cache/pip/wheels/5c/ba/f0/4c8b421be72d4f2d1a93233c2f6f591e7d8b0bda05a1f4616f
Successfully built kmc2


In [0]:
seeding = kmc2.kmc2(np.array(traindes).reshape(-1,128), codebooksize) 
Kmeas = MiniBatchKMeans(codebooksize, init=seeding).fit(np.array(traindes).reshape(-1,128))
codebook = Kmeas.clustercenters

NameError: ignored

In [0]:
# import packages here
import cv2
import matplotlib.pyplot as plt
import glob
import pickle
import os
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.svm import LinearSVC
import copy
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import svm
import numpy as np
import torch
import random
import sys
from scipy.sparse import issparse
from sklearn.utils import gen_batches
from sklearn.svm import SVC
import sklearn
import scipy.cluster.vq as vq
import pandas as pd
device_gpu = torch.device('cuda')
device_cpu = torch.device('cpu')
import subprocess
use_cuda = torch.cuda.is_available()
from sklearn.decomposition import PCA
import kmc2
dtype = 'float32' if use_cuda else 'float64'
torchtype = {'float32': torch.float32, 'float64': torch.float64}
dataroot="/content/gdrive/My Drive/"
csvpath=os.path.join(dataroot,"Label2Names.csv")
df_data=pd.read_csv(csvpath,header=None)

dataroottrain=os.path.join(dataroot,"train")
dataroottest=os.path.join(dataroot,"testAll_v2")


In [0]:


##################dataload train
def loadtrain(dataroottrain):
    trainlabel=[]
    traindata=[]


    #import pdb;pdb.set_trace()
    for classname in os.listdir(dataroottrain):
          if classname[0]=='.':
                continue
          classpath=os.path.join(dataroottrain,classname)
          #import pdb;pdb.set_trace()
          if classname=="BACKGROUND_Google":
            labelind=102
          else:
            labelind=(df_data.index[df_data[1]==classname]+1).tolist()[0]  #import pdb;pdb.set_trace()

          for imgname in os.listdir(classpath):
            trainlabel.append(labelind)
            imgpath=os.path.join(classpath,imgname)

            img=cv2.imread(imgpath)
            img=cv2.resize(img,(256,256))
            #descriptor=extract_denseSIFT(img)
            traindata.append(img)
          print("Done "+classname)
    #traindata=np.array(traindata)
    #trainlabel=np.array(trainlabel)
    return traindata,trainlabel
##################dataload test
def loadtest(dataroottest):
    testlabel=[]
    testdata=[]
    testsort=sorted(os.listdir(dataroottest))
    for classname in testsort:
          if classname[0]=='.':
                continue
          testlabel.append(classname)
          classpath=os.path.join(dataroottest,classname)
          #import pdb;pdb.set_trace()
              
          imgpath=os.path.join(classpath)

          img=cv2.imread(imgpath)
          img=cv2.resize(img,(256,256))
          #descriptor=extract_denseSIFT(img)
          testdata.append(img)
          print("Done "+classname)
    #testdata=np.array(testdata)
    return testdata,testlabel
#import pdb;pdb.set_trace()
#########################################################################################################
def extract_sift_descriptors(img):
    """
    Input BGR numpy array
    Return SIFT descriptors for an image
    Return None if there's no descriptor detected
    """
    gray=img
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors


def extract_DenseSift_descriptors(img):
    """
    Input BGR numpy array
    Return Dense SIFT descriptors for an image
    Return None if there's no descriptor detected
    """
    #import pdb;pdb.set_trace()
    gray=img
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()

    # opencv docs DenseFeatureDetector
    # opencv 2.x code
    #dense.setInt('initXyStep',8) # each step is 8 pixel
    #dense.setInt('initImgBound',8)
    #dense.setInt('initFeatureScale',16) # each grid is 16*16
    disft_step_size = DSIFT_STEP_SIZE
    keypoints = [cv2.KeyPoint(x, y, disft_step_size)
            for y in range(0, gray.shape[0], disft_step_size)
                for x in range(0, gray.shape[1], disft_step_size)]

    keypoints, descriptors = sift.compute(gray, keypoints)
    #import pdb;pdb.set_trace()
    
   
    #keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    return [keypoints, descriptors]


def build_codebook(X, voc_size):
    """
    Inupt a list of feature descriptors
    voc_size is the "K" in K-means, k is also called vocabulary size
    Return the codebook/dictionary
    """
    #features = np.vstack((descriptor for descriptor in X))
    #features=torch.from_numpy(features)
    from sklearn.cluster import MiniBatchKMeans 
    codebooksize=voc_size
    #import pdb;pdb.set_trace()
    seeding = kmc2.kmc2(np.array(X).reshape(-1,np.array(X).shape[2]), codebooksize) 
    Kmeas = MiniBatchKMeans(codebooksize, init=seeding,init_size=2*1024).fit(np.array(X).reshape(-1,np.array(X).shape[2]))
    codebook = Kmeas.cluster_centers_
    #codebook,_ = KMeans_GPU(features,200)
    #codebook =codebook.cpu().numpy();
    
    return codebook

from sklearn.decomposition import SparseCoder
def input_vector_encoder(feature, codebook):
    """
    Input all the local feature of the image
    Pooling (encoding) by codebook and return
    """
    #import pdb;pdb.set_trace()
    #coder = SparseCoder(dictionary=codebook, transform_algorithm='threshold')
    #xx = coder.transform(feature)
    code, _ = vq.vq(feature, codebook)
    
    #import pdb;pdb.set_trace()
    word_hist, bin_edges = np.histogram(code, bins=range(codebook.shape[0] + 1), normed=True)
    return word_hist
def build_spatial_pyramid_dc(image, descriptor, level):
    """
    Rebuild the descriptors according to the level of pyramid
    """
    assert 0 <= level <= 2, "Level Error"
    step_size = DSIFT_STEP_SIZE
    s =8
    assert s == step_size, "step_size must equal to DSIFT_STEP_SIZE\
                            in utils.extract_DenseSift_descriptors()"
    h = image.shape[0] /8
    w = image.shape[1] / 8
    #import pdb;pdb.set_trace()
    if h%1!=0:
        h+=1
    if w%1!=0:
        w+=1
    h=int(h)
    w=int(w)
    
    idx_crop = np.array(range(len(descriptor))).reshape(h,w)
    size = idx_crop.itemsize
    height, width = idx_crop.shape
    bh, bw = 8,8
    #import pdb;pdb.set_trace()  
    shape = (4, 4, 8, 8)
    strides =np.array([2048, 64, 256, 8])
    
    #import pdb;pdb.set_trace()  
    shape=np.array(shape).astype(int)
    crops = np.lib.stride_tricks.as_strided(idx_crop, shape=shape, strides=strides)
    import pdb;pdb.set_trace() 
    des_idxs = [col_block.flatten().tolist() for row_block in crops
                for col_block in row_block]
    pyramid = []
    for idxs in des_idxs:
        pyramid.append([descriptor[idx] for idx in idxs])
    #import pdb;pdb.set_trace()
    return pyramid
def build_spatial_pyramid(image, descriptor, level):
    """
    Rebuild the descriptors according to the level of pyramid
    """
    assert 0 <= level <= 2, "Level Error"
    step_size = DSIFT_STEP_SIZE
    s =8
    assert s == step_size, "step_size must equal to DSIFT_STEP_SIZE\
                            in utils.extract_DenseSift_descriptors()"
    h = image.shape[0] / step_size
    w = image.shape[1] / step_size
    #import pdb;pdb.set_trace()
    if h%1!=0:
        h+=1
    if w%1!=0:
        w+=1
    h=int(h)
    w=int(w)
    
    idx_crop = np.array(range(len(descriptor))).reshape(int(h),int(w))
    size = idx_crop.itemsize
    height, width = idx_crop.shape
    bh, bw =16, 16
    shape = (2, 2, 16, 16)
    strides = np.array([2048*2, 64*2, 256, 8])
      
    shape=np.array(shape).astype(int)
    crops = np.lib.stride_tricks.as_strided(idx_crop, shape=shape, strides=strides)
    #import pdb;pdb.set_trace()  
    des_idxs = [col_block.flatten().tolist() for row_block in crops
                for col_block in row_block]
    pyramid = []
    for idxs in des_idxs:
        pyramid.append([descriptor[idx] for idx in idxs])
    #import pdb;pdb.set_trace()
    return pyramid

def spatial_pyramid_matching(image, descriptor, codebook, level):
    pyramid = []
    if level == 0:
        
        pyramid +=[descriptor.tolist()]
        code = [input_vector_encoder(crop, codebook) for crop in pyramid]
        return np.asarray(code).flatten()
    if level == 1:
        #import pdb;pdb.set_trace()    
        pyramid +=[descriptor.tolist()]
        pyramid +=  build_spatial_pyramid(image, descriptor, level=0)
        code = [input_vector_encoder(crop, codebook) for crop in pyramid]
        code_level_0 = 0.5 * np.asarray(code[0]).flatten()
        code_level_1 = 0.5 * np.asarray(code[1:]).flatten()
        return np.concatenate((code_level_0, code_level_1))
    if level == 2:
        pyramid += [descriptor.tolist()]
        pyramid +=  build_spatial_pyramid(image, descriptor, level=0)
        pyramid +=build_spatial_pyramid_dc(image, descriptor, level=1)
        code = [input_vector_encoder(crop, codebook) for crop in pyramid]
        #import pdb;pdb.set_trace()
        code_level_0 = 0.25 * np.asarray(code[0]).flatten()
        code_level_1 = 0.25 * np.asarray(code[1:5]).flatten()
        code_level_2 = 0.5 * np.asarray(code[5:]).flatten()
        return np.concatenate((code_level_0, code_level_1, code_level_2))
def get_label(path='./cifar10/train/labels.txt'):
    """ Get cifar10 class label"""
    with open(path,'r') as f:
        names = f.readlines()
    names = [n.strip() for n in names]
    return names

def svm_classifier(x_train, y_train, x_test=None, y_test=None,level=1):
    best=0
    ##############################################
    '''
    for c in np.arange(0.0001, 0.1, 0.00198):
        clf = LinearSVC(random_state=0, C=c)
        #import pdb;pdb.set_trace()
        clf.fit(x_train, y_train)
        predict = clf.predict(x_test)

    predict = clf.predict(x_test)
    return predict
    '''
    ##############################################
    C_range = 10.0 ** np.arange(1, 4)
    #gamma_range = 10.0 ** np.arange(-3, 3)
    param_grid = dict(C=C_range.tolist())

    # Grid search for C, gamma, 5-fold CV
    print("Tuning hyper-parameters\n")
    clf = GridSearchCV(LinearSVC(random_state=0), param_grid, cv=5, n_jobs=-2)
    clf.fit(x_train, y_train)
    print(clf.best_params_)
    predict=clf.predict(x_test)
    return predict
    
# form histogram with Spatial Pyramid Matching upto level L with codebook kmeans and k codewords

def histogramIntersection(M, N):
    m = M.shape[0]
    n = N.shape[0]
    result = np.zeros((m,n))
    for i in range(m):
        for j in range(n):
            temp = np.sum(np.minimum(M[i], N[j]))
            result[i][j] = temp
    return result

def gramSVM(total_train_feature,total_test_feature,train_labels):
    C_range = 10.0 ** np.arange(-4, 3)
    #gamma_range = 10.0 ** np.arange(-3, 3)
    param_grid = dict(C=C_range.tolist())
    gramMatrix = histogramIntersection(total_train_feature, total_train_feature)
    # scaler.fit(gramMatrix)
    # gramMatrix = scaler.transform(gramMatrix)
    clf = GridSearchCV( svm.SVC(kernel='precomputed'), param_grid, cv=5, n_jobs=-2)
    clf.fit(gramMatrix, train_labels)
    predictMatrix = histogramIntersection(total_test_feature, total_train_feature)
    # predictMatrix = scaler.transform(predictMatrix)
    SVMResults = clf.predict(predictMatrix)
    print(clf.best_params_)
    return SVMResults
def PCA_(des):
    pca = PCA(n_components=100, svd_solver='randomized',whiten=True)
    des=pca.fit_transform(des)
    return des
######################################################Main
def VLAD(X,codebook):

    predictedLabels,_ = vq.vq(X,codebook)
    centers = codebook
    #labels=visualDictionary.labels_
    k=1024
   
    m,d = X.shape
    V=np.zeros([k,d])
    #computing the differences

    # for all the clusters (visual words)
    for i in range(k):
        # if there is at least one descriptor in that cluster
        if np.sum(predictedLabels==i)>0:
            # add the diferences
            V[i]=np.sum(X[predictedLabels==i,:]-centers[i],axis=0)
    

    V = V.flatten()
    # power normalization, also called square-rooting normalization
    V = np.sign(V)*np.sqrt(np.abs(V))

    # L2 normalization

    V = V/np.sqrt(np.dot(V,V))
    return V

In [0]:
if not os.path.exists(os.path.join(dataroot,"traindata.pkl")):
    print("Make data")
    train_data,train_label=loadtrain(dataroottrain)
    print("Done train")
    test_data,test_label=loadtest(dataroottest)
    print("Done test")
    #import pdb;pdb.set_trace()
    with open(os.path.join(dataroot,"traindata.pkl"), 'wb') as f:
        pickle.dump(train_data, f)
    with open(os.path.join(dataroot,"testdata.pkl"), 'wb') as f:
        pickle.dump(test_data, f)
    with open(os.path.join(dataroot,"trainlabel.pkl"), 'wb') as f:
        pickle.dump(train_label, f)
    with open(os.path.join(dataroot,"testlabel.pkl"), 'wb') as f:
        pickle.dump(test_label, f)
else:
    print("Load data")
    with open(os.path.join(dataroot,"traindata.pkl"), 'rb') as f:
        train_data=pickle.load(f)
    with open(os.path.join(dataroot,"testdata.pkl"), 'rb') as f:
        test_data=pickle.load(f)
    with open(os.path.join(dataroot,"trainlabel.pkl"), 'rb') as f:
        train_label=pickle.load(f)
    with open(os.path.join(dataroot,"testlabel.pkl"), 'rb') as f:
        test_label=pickle.load(f)     

DSIFT_STEP_SIZE = 8
#import pdb;pdb.set_trace()
train_data=np.array(train_data)
train_label=np.array(train_label)
train_data=train_data[train_label!=102]
train_label=train_label[train_label!=102]
from sklearn.model_selection import train_test_split
#train_data,Val_data, train_label, Val_label = train_test_split(train_data, train_label, test_size=0.1, random_state=42)

Load data


In [0]:

print(len(train_data),len(train_label))


3030 3030


In [0]:
from sklearn.metrics import accuracy_score
def formTrainingSetHistogram(x_train, codebook, k):
    train_hist = []
    for i in range(len(x_train)):
        data = copy.deepcopy(x_train[i])
        predict,_= vq.vq(x_train[i],codebook)
        #import pdb;pdb.set_trace()
        train_hist.append(np.bincount(predict, minlength=k).reshape(1,-1).ravel())
        
    return np.array(train_hist)
def make_kp_arr(x_kp):
  x_arr=[]
  for keypoints in x_kp:
    i_kp=[]
    for keypoint in keypoints:
      xy=[]
      for i in range(2):
        xy.append(keypoint.pt[i])
      x_arr.append(xy)  
  x_arr=np.array(x_arr).astype("float32")
  return  x_arr
def normalize_extension(x_train_des,x_train_kp):
  img_num=np.array(x_train_des).shape[0]
  des_num=np.array(x_train_des).shape[1]
  x_train_des=np.transpose(x_train_des,(2,0,1))
  x_train_des=x_train_des.reshape(x_train_des.shape[0],x_train_des.shape[1]*x_train_des.shape[2])
  ###########normalize
  sqrt=np.sqrt(np.square(x_train_des).sum(0))
  sqrt[sqrt==0]=1e-5
  nor=1./sqrt
  nor=nor.reshape(1,-1)
  x_train_des=x_train_des*nor

  ############
  ###########PCA
  # encoder.projection = diag(1./sqrt(d(1:m))) * V(:,1:m)' ;
  '''
  mean=x_train_des.mean(1).reshape(-1,1)
  x=x_train_des-mean
  X = np.dot(x,np.transpose(x))/ x.shape[1]
  #import pdb;pdb.set_trace()
  import scipy.linalg as la
  D, V = la.eig(X)
  d=D
  d=np.argsort(d)
  d=d[::-1]
  m=100
  perm=np.array(range(127,-1,-1))
  d=d*0.001*np.max(d)
  #import pdb;pdb.set_trace()
  sqrt_=np.sqrt(d[:m])
  sqrt_[sqrt_==0]=1e-5
  V=V[:,perm]
  projection=np.dot(np.diag(1./sqrt_),np.transpose(V[:,:m]))'''
  #import pdb;pdb.set_trace()
  #import pdb;pdb.set_trace()
  ###########################################################
  mean=x_train_des.mean(1).reshape(-1,1)
  x_train_des=np.dot(projection,x_train_des-mean)
  ########################################################
  sqrt=np.sqrt(np.square(x_train_des).sum(0))
  sqrt[sqrt==0]=1e-5
  nor=1./sqrt
  nor=nor.reshape(1,-1)
  x_train_des=x_train_des*nor
  #########################################################
  ################geomatric_extension
  x_train_kp=x_train_kp-256/2
  x_train_kp=x_train_kp/256
  x_train_kp=np.transpose(x_train_kp,(1,0))
  x_train_des=np.concatenate((x_train_des,x_train_kp))
  #########################################################
  sqrt=np.sqrt(np.square(x_train_des).sum(0))
  sqrt[sqrt==0]=1e-12
  nor=1./sqrt
  nor=nor.reshape(1,-1)
  x_train_des=x_train_des*nor
  #########################################################
  x_train_des=np.transpose(x_train_des,(1,0))
  x_train_des=x_train_des.reshape(img_num,des_num,102)
  
  return x_train_des
for i in range(0,3):
    print("SPM START%d"%i) 
    if not os.path.exists(os.path.join(dataroot,"/SVMlevel%d.pkl"%i)):
        PYRAMID_LEVEL=i
        name=7
        #import pdb;pdb.set_trace()
        if not os.path.exists(os.path.join(dataroot,'spm_lv%d_trainfeature_%d.pkl'%(0,name))):
            
            x_train_feature = [extract_DenseSift_descriptors(img) for img in train_data]
            
            x_test_feature = [extract_DenseSift_descriptors(img) for img in Val_data]
        
            x_train_kp, x_train_des = zip(*x_train_feature)
            x_test_kp, x_test_des = zip(*x_test_feature)
            print("Done Dense SIFT")
            #import pdb;pdb.set_trace()
            #import pdb;pdb.set_trace()
            x_train_kp=make_kp_arr(x_train_kp)
            x_test_kp=make_kp_arr(x_test_kp)
            
            with open(os.path.join(dataroot,'spm_lv%d_trainfeature_%d.pkl'%(0,name)),'wb') as f:
                pickle.dump(x_train_des, f)
            with open(os.path.join(dataroot,'spm_lv%d_valfeature_%d.pkl'%(0,name)),'wb') as f:
                pickle.dump(x_test_des, f)
            with open(os.path.join(dataroot,'spm_lv%d_trainkp_%d.pkl'%(0,name)),'wb') as f:
                pickle.dump(x_train_kp, f)
            with open(os.path.join(dataroot,'spm_lv%d_valkp_%d.pkl'%(0,name)),'wb') as f:
                pickle.dump(x_test_kp, f)
        else:
            with open(os.path.join(dataroot,'spm_lv%d_trainfeature_%d.pkl'%(0,name)),'rb') as f:
                x_train_des=pickle.load(f)
                
            with open(os.path.join(dataroot,'spm_lv%d_valfeature_%d.pkl'%(0,name)),'rb') as f:
                x_test_des=pickle.load(f)  
            
            with open(os.path.join(dataroot,'spm_lv%d_trainkp_%d.pkl'%(0,name)),'rb') as f:
                x_train_kp=pickle.load(f)
                
            with open(os.path.join(dataroot,'spm_lv%d_valkp_%d.pkl'%(0,name)),'rb') as f:
                x_test_kp=pickle.load(f)
        #import pdb;pdb.set_trace()        
        #x_train_des=[PCA_(i) for i in x_train_des]
        #x_test_des=[PCA_(i) for i in x_test_des]
        
        ###########################################################
        #dot=1./max(1e-5,np.sqrt(np.sum(np.square(descriptors))))
        
        x_train_des=normalize_extension(x_train_des,x_train_kp)
        x_test_des=normalize_extension(x_test_des,x_test_kp)
        #################################
        #import pdb;pdb.set_trace()
        
        
        
        
        
        if not os.path.exists(os.path.join(dataroot,'spm_lv%d_codebook_200_%d.pkl'%(0,name))):
            codebook = build_codebook(x_train_des, 1024)
            with open(os.path.join(dataroot,'spm_lv%d_codebook_200_%d.pkl'%(0,name)),'wb') as f:
                pickle.dump(codebook, f)
        else:
            with open(os.path.join(dataroot,'spm_lv%d_codebook_200_%d.pkl'%(0,name)),'rb') as f:
                codebook=pickle.load(f)
            print("Load Codebook")
        #import pdb;pdb.set_trace()
        
        x_spm_train=[VLAD(i,codebook)for i in x_train_des]  
        x_spm_test=[VLAD(i,codebook) for i in x_test_des] 
        x_spm_train = np.array(x_spm_train)
        x_spm_test = np.array(x_spm_test)

        '''
        #x_train_des=[PCA_(i) for i in x_train_des]
        #if not os.path.exists('./SPMfeaturelevel%d_train%d.pkl'%(i,i)):
        x_spm_train = [spatial_pyramid_matching(train_data[i],
                                                x_train_des[i],
                                                codebook,
                                                level=PYRAMID_LEVEL)
                                                for i in range(len(train_data))]
        x_spm_train = np.asarray(x_spm_train)
        #    with open('./SPMfeaturelevel%d_train%d.pkl'%(i,i),'wb') as f:
        #        pickle.dump(x_spm_train, f)
        #else:
        #    with open('./SPMfeaturelevel%d_train%d.pkl'%(i,i),'rb') as f:
        #        x_spm_train=pickle.load(f)
        print("Load SPM")
        
        #if not os.path.exists('./SPMfeaturelevel%d_test%d.pkl'%(i,i)):
        x_spm_test = [spatial_pyramid_matching(Val_data[i],
                                               x_test_des[i],
                                               codebook,
                                               level=PYRAMID_LEVEL) for i in range(len(Val_data))]
                                          
        x_spm_test = np.asarray(x_spm_test)'''
        '''
        
        if not os.path.exists('./SPMfeaturelevel%d_train%d.pkl'%(i,i)):
            x_spm_train = getHistogramSPM(i, train_data,codebook, 200)
            x_spm_test = getHistogramSPM(i, test_data,codebook, 200)
            with open('./SPMfeaturelevel%d_train%d.pkl'%(i,i),'wb') as f:
                pickle.dump(x_spm_train, f)
            with open('./SPMfeaturelevel%d_test%d.pkl'%(i,i),'wb') as f:
                pickle.dump(x_spm_test, f)
        
        else:
            with open('./SPMfeaturelevel%d_train%d.pkl'%(i,i),'rb') as f:
                x_spm_train=pickle.load(f)
            with open('./SPMfeaturelevel%d_test%d.pkl'%(i,i),'rb') as f:
                x_spm_test=pickle.load(f)
        #    with open('./SPMfeaturelevel%d_test%d.pkl'%(i,i),'wb') as f:
        #        pickle.dump(x_spm_test, f)
        #else:
        #    with open('./SPMfeaturelevel%d_test%d.pkl'%(i,i),'rb') as f:
        #        x_spm_test=pickle.load(f) 
        '''
        print("Load SPM")    
        
        
            
        #import pdb;pdb.set_trace()
        '''
        pca = PCA(n_components=100, svd_solver='randomized',whiten=True)
        
        x_spm_train=pca.fit_transform(x_spm_train)
        x_spm_test=pca.fit_transform(x_spm_test)'''
        print ("Done PCA")
        #SVM_Classify(x_spm_train,train_label,train_label,test_label,i)
        #predict=SVM_Classify(x_spm_train, train_label, x_spm_test, test_label,i)
        #import pdb;pdb.set_trace()
        predict =svm_classifier(x_spm_train, train_label,x_spm_test )          ############Linear SVM
        
        #predict=gramSVM(x_spm_train,x_spm_test,train_label)                      #########intersection
        print(accuracy_score(Val_label,predict))
       

SPM START0


KeyboardInterrupt: ignored

# 새 섹션

In [0]:
from sklearn.metrics import accuracy_score
def formTrainingSetHistogram(x_train, codebook, k):
    train_hist = []
    for i in range(len(x_train)):
        data = copy.deepcopy(x_train[i])
        predict,_= vq.vq(x_train[i],codebook)
        #import pdb;pdb.set_trace()
        train_hist.append(np.bincount(predict, minlength=k).reshape(1,-1).ravel())
        
    return np.array(train_hist)
def make_kp_arr(x_kp):
  x_arr=[]
  for keypoints in x_kp:
    i_kp=[]
    for keypoint in keypoints:
      xy=[]
      for i in range(2):
        xy.append(keypoint.pt[i])
      x_arr.append(xy)  
  x_arr=np.array(x_arr).astype("float32")
  return  x_arr
def normalize_extension(x_train_des,x_train_kp):
  img_num=np.array(x_train_des).shape[0]
  des_num=np.array(x_train_des).shape[1]
  x_train_des=np.transpose(x_train_des,(2,0,1))
  x_train_des=x_train_des.reshape(x_train_des.shape[0],x_train_des.shape[1]*x_train_des.shape[2])
  ###########normalize
  sqrt=np.sqrt(np.square(x_train_des).sum(0))
  sqrt[sqrt==0]=1e-5
  nor=1./sqrt
  nor=nor.reshape(1,-1)
  x_train_des=x_train_des*nor

  ############
  ###########PCA
  # encoder.projection = diag(1./sqrt(d(1:m))) * V(:,1:m)' ;
  '''
  mean=x_train_des.mean(1).reshape(-1,1)
  new_matrix=x_train_des-mean
  covariance_matrix = np.cov(np.transpose(new_matrix))
  w,v = LA.eig(covariance_matrix)
  U_Truncated = v[:,:100]
  U_Truncated_Transpose = np.transpose(U_Truncated)
  new_dimension_matrix = np.dot(U_Truncated_Transpose,np.transpose(new_matrix))
  newData = np.transpose(new_dimension_matrix)
  val=(-w).argsort()[:100]
  result=v[...,val]'''
  #import pdb;pdb.set_trace()
  #import pdb;pdb.set_trace()
  ###########################################################
  mean=x_train_des.mean(1).reshape(-1,1)
  x_train_des=x_train_des-mean
  ########################################################
  sqrt=np.sqrt(np.square(x_train_des).sum(0))
  sqrt[sqrt==0]=1e-5
  nor=1./sqrt
  nor=nor.reshape(1,-1)
  x_train_des=x_train_des*nor
  #########################################################
  ################geomatric_extension
  
  x_train_kp=x_train_kp-256/2
  x_train_kp=x_train_kp/256
  x_train_kp=np.transpose(x_train_kp,(1,0))
  x_train_des=np.concatenate((x_train_des,x_train_kp))
  #########################################################
  sqrt=np.sqrt(np.square(x_train_des).sum(0))
  sqrt[sqrt==0]=1e-12
  nor=1./sqrt
  nor=nor.reshape(1,-1)
  x_train_des=x_train_des*nor
  #########################################################
  x_train_des=np.transpose(x_train_des,(1,0))
  x_train_des=x_train_des.reshape(img_num,des_num,130)
  
  return x_train_des
for i in range(0,3):
    print("SPM START%d"%i) 
    if not os.path.exists(os.path.join(dataroot,"/SVMlevel%d.pkl"%i)):
        PYRAMID_LEVEL=i
        name=9
        #import pdb;pdb.set_trace()
        if not os.path.exists(os.path.join(dataroot,'spm_lv%d_trainfeature_%d.pkl'%(0,name)))or not os.path.exists(os.path.join(dataroot,'spm_lv%d_testfeature_%d.pkl'%(0,name))):
            
            x_train_feature = [extract_DenseSift_descriptors(img) for img in train_data]
            
            x_test_feature = [extract_DenseSift_descriptors(img) for img in test_data]
        
            x_train_kp, x_train_des = zip(*x_train_feature)
            x_test_kp, x_test_des = zip(*x_test_feature)
            print("Done Dense SIFT")
            #import pdb;pdb.set_trace()
            #import pdb;pdb.set_trace()
            x_train_kp=make_kp_arr(x_train_kp)
            x_test_kp=make_kp_arr(x_test_kp)
            
            with open(os.path.join(dataroot,'spm_lv%d_trainfeature_%d.pkl'%(0,name)),'wb') as f:
                pickle.dump(x_train_des, f)
            with open(os.path.join(dataroot,'spm_lv%d_testfeature_%d.pkl'%(0,name)),'wb') as f:
                pickle.dump(x_test_des, f)
            with open(os.path.join(dataroot,'spm_lv%d_trainkp_%d.pkl'%(0,name)),'wb') as f:
                pickle.dump(x_train_kp, f)
            with open(os.path.join(dataroot,'spm_lv%d_testkp_%d.pkl'%(0,name)),'wb') as f:
                pickle.dump(x_test_kp, f)
        else:
            with open(os.path.join(dataroot,'spm_lv%d_trainfeature_%d.pkl'%(0,name)),'rb') as f:
                x_train_des=pickle.load(f)
                
            with open(os.path.join(dataroot,'spm_lv%d_testfeature_%d.pkl'%(0,name)),'rb') as f:
                x_test_des=pickle.load(f)  
            
            with open(os.path.join(dataroot,'spm_lv%d_trainkp_%d.pkl'%(0,name)),'rb') as f:
                x_train_kp=pickle.load(f)
                
            with open(os.path.join(dataroot,'spm_lv%d_testkp_%d.pkl'%(0,name)),'rb') as f:
                x_test_kp=pickle.load(f)
        #import pdb;pdb.set_trace()        
        #x_train_des=[PCA_(i) for i in x_train_des]
        #x_test_des=[PCA_(i) for i in x_test_des]
        
        ###########################################################
        #dot=1./max(1e-5,np.sqrt(np.sum(np.square(descriptors))))
        
        x_train_des=normalize_extension(x_train_des,x_train_kp)
        x_test_des=normalize_extension(x_test_des,x_test_kp)
        #################################
        #import pdb;pdb.set_trace()
        
        
        
        
        
        if not os.path.exists(os.path.join(dataroot,'spm_lv%d_codebook_200_%d.pkl'%(0,name))):
            codebook = build_codebook(x_train_des, 1024)
            with open(os.path.join(dataroot,'spm_lv%d_codebook_200_%d.pkl'%(0,name)),'wb') as f:
                pickle.dump(codebook, f)
        else:
            with open(os.path.join(dataroot,'spm_lv%d_codebook_200_%d.pkl'%(0,name)),'rb') as f:
                codebook=pickle.load(f)
            print("Load Codebook")
        #import pdb;pdb.set_trace()
        
        x_spm_train=[VLAD(i,codebook)for i in x_train_des]  
        x_spm_test=[VLAD(i,codebook) for i in x_test_des] 
        x_spm_train = np.array(x_spm_train)
        x_spm_test = np.array(x_spm_test)
        '''
        #x_train_des=[PCA_(i) for i in x_train_des]
        #if not os.path.exists('./SPMfeaturelevel%d_train%d.pkl'%(i,i)):
        x_spm_train = [spatial_pyramid_matching(train_data[i],
                                                x_train_des[i],
                                                codebook,
                                                level=PYRAMID_LEVEL)
                                                for i in range(len(train_data))]
        x_spm_train = np.asarray(x_spm_train)
        #    with open('./SPMfeaturelevel%d_train%d.pkl'%(i,i),'wb') as f:
        #        pickle.dump(x_spm_train, f)
        #else:
        #    with open('./SPMfeaturelevel%d_train%d.pkl'%(i,i),'rb') as f:
        #        x_spm_train=pickle.load(f)
        print("Load SPM")
        
        #if not os.path.exists('./SPMfeaturelevel%d_test%d.pkl'%(i,i)):
        x_spm_test = [spatial_pyramid_matching(Val_data[i],
                                               x_test_des[i],
                                               codebook,
                                               level=PYRAMID_LEVEL) for i in range(len(Val_data))]
                                          
        x_spm_test = np.asarray(x_spm_test)'''
        '''
        
        if not os.path.exists('./SPMfeaturelevel%d_train%d.pkl'%(i,i)):
            x_spm_train = getHistogramSPM(i, train_data,codebook, 200)
            x_spm_test = getHistogramSPM(i, test_data,codebook, 200)
            with open('./SPMfeaturelevel%d_train%d.pkl'%(i,i),'wb') as f:
                pickle.dump(x_spm_train, f)
            with open('./SPMfeaturelevel%d_test%d.pkl'%(i,i),'wb') as f:
                pickle.dump(x_spm_test, f)
        
        else:
            with open('./SPMfeaturelevel%d_train%d.pkl'%(i,i),'rb') as f:
                x_spm_train=pickle.load(f)
            with open('./SPMfeaturelevel%d_test%d.pkl'%(i,i),'rb') as f:
                x_spm_test=pickle.load(f)
        #    with open('./SPMfeaturelevel%d_test%d.pkl'%(i,i),'wb') as f:
        #        pickle.dump(x_spm_test, f)
        #else:
        #    with open('./SPMfeaturelevel%d_test%d.pkl'%(i,i),'rb') as f:
        #        x_spm_test=pickle.load(f) 
        '''
        print("Load SPM")    
        
        
            
        #import pdb;pdb.set_trace()
        
        pca = PCA(n_components=1024,whiten=True)
        
        x_spm_train=pca.fit_transform(x_spm_train)
        x_spm_test=pca.transform(x_spm_test)
        print ("Done PCA")
        #SVM_Classify(x_spm_train,train_label,train_label,test_label,i)
        #predict=SVM_Classify(x_spm_train, train_label, x_spm_test, test_label,i)
        #import pdb;pdb.set_trace()
        predict =svm_classifier(x_spm_train, train_label,x_spm_test )          ############Linear SVM
        
        #predict=gramSVM(x_spm_train,x_spm_test,train_label)                      #########intersection
        #print(accuracy_score(Val_label,predict))

        predict=predict.reshape(-1,1)
        
        test_label=np.array(test_label)
        test_label=test_label.reshape(-1,1)
        #import pdb;pdb.set_trace()
        total_result=np.hstack([test_label,predict])
        
        df = pd.DataFrame(total_result,columns=["Id","Category"])
        df.to_csv("./gdrive/My Drive/result_dchan_level.csv",index=False,header=True)

SPM START0
Load Codebook
Load SPM
Done PCA
Tuning hyper-parameters



/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


{'C': 10.0}
SPM START1
Load Codebook


KeyboardInterrupt: ignored

In [0]:
! kaggle competitions submit -c 2019-ml-finalproject -f result_dchan_level.csv -m DCHAN_FINAL

In [0]:
test_data

[array([[[ 55,  52,  47],
         [146, 144, 139],
         [144, 144, 140],
         ...,
         [168, 173, 172],
         [166, 171, 170],
         [166, 170, 170]],
 
        [[ 46,  43,  38],
         [122, 120, 115],
         [127, 127, 123],
         ...,
         [170, 175, 174],
         [159, 164, 163],
         [156, 161, 160]],
 
        [[ 61,  58,  53],
         [126, 124, 119],
         [129, 129, 125],
         ...,
         [173, 178, 177],
         [169, 174, 173],
         [168, 173, 172]],
 
        ...,
 
        [[ 46,  38,  31],
         [139, 133, 127],
         [115, 112, 109],
         ...,
         [ 98, 103, 102],
         [114, 119, 118],
         [123, 127, 126]],
 
        [[ 50,  43,  36],
         [139, 133, 127],
         [124, 121, 117],
         ...,
         [ 93,  98,  97],
         [116, 120, 119],
         [126, 131, 130]],
 
        [[ 55,  48,  40],
         [128, 123, 117],
         [110, 107, 104],
         ...,
         [ 94,  99,  98],
  

In [0]:
def formTrainingSetHistogram(x_train, codebook, k):
    train_hist = []
    for i in range(len(x_train)):
        data = copy.deepcopy(x_train[i])
        predict,_= vq.vq(x_train[i],codebook)
        #import pdb;pdb.set_trace()
        train_hist.append(np.bincount(predict, minlength=k).reshape(1,-1).ravel())
        
    return np.array(train_hist)
for i in range(2,3):
    print("SPM START%d"%i) 
    if not os.path.exists(os.path.join(dataroot,"/SVMlevel%d.pkl"%i)):
        PYRAMID_LEVEL=i
        name=6
        #import pdb;pdb.set_trace()
        if not os.path.exists(os.path.join(dataroot,'spm_lv%d_trainfeature_%d.pkl'%(0,name))):
            
            x_train_feature = [extract_DenseSift_descriptors(img) for img in train_data]
            
            x_test_feature = [extract_DenseSift_descriptors(img) for img in test_data]
        
            x_train_kp, x_train_des = zip(*x_train_feature)
            x_test_kp, x_test_des = zip(*x_test_feature)
            print("Done Dense SIFT")
            
            #import pdb;pdb.set_trace()
            with open(os.path.join(dataroot,'spm_lv%d_trainfeature_%d.pkl'%(0,name)),'wb') as f:
                pickle.dump(x_train_des, f)
            with open(os.path.join(dataroot,'spm_lv%d_testfeature_%d.pkl'%(0,name)),'wb') as f:
                pickle.dump(x_test_des, f)
        else:
            with open(os.path.join(dataroot,'spm_lv%d_trainfeature_%d.pkl'%(0,name)),'rb') as f:
                x_train_des=pickle.load(f)
                
            with open(os.path.join(dataroot,'spm_lv%d_testfeature_%d.pkl'%(0,name)),'rb') as f:
                x_test_des=pickle.load(f)
        #import pdb;pdb.set_trace()        
        #x_train_des=[PCA_(i) for i in x_train_des]
        #x_test_des=[PCA_(i) for i in x_test_des]
        
       
        if not os.path.exists(os.path.join(dataroot,'spm_lv%d_codebook_200_%d.pkl'%(0,name))):
            codebook = build_codebook(x_train_des, 200)
            with open(os.path.join(dataroot,'spm_lv%d_codebook_200_%d.pkl'%(0,name)),'wb') as f:
                pickle.dump(codebook, f)
        else:
            with open(os.path.join(dataroot,'spm_lv%d_codebook_200_%d.pkl'%(0,name)),'rb') as f:
                codebook=pickle.load(f)
            print("Load Codebook")
        '''
        x_spm_train=[VLAD(i,codebook)for i in x_train_des]  
        x_spm_test=[VLAD(i,codebook) for i in x_test_des] 
        x_spm_train = np.array(x_spm_train)
        x_spm_test = np.array(x_spm_test)
        '''
        x_spm_train=formTrainingSetHistogram(x_train_des,codebook,200)
        x_spm_test=formTrainingSetHistogram(x_test_des,codebook,200)
        
        #x_train_des=[PCA_(i) for i in x_train_des]
        #if not os.path.exists('./SPMfeaturelevel%d_train%d.pkl'%(i,i)):
        x_spm_train = [spatial_pyramid_matching(train_data[i],
                                                x_train_des[i],
                                                codebook,
                                                level=PYRAMID_LEVEL)
                                                for i in range(len(train_data))]
        x_spm_train = np.asarray(x_spm_train)
        #    with open('./SPMfeaturelevel%d_train%d.pkl'%(i,i),'wb') as f:
        #        pickle.dump(x_spm_train, f)
        #else:
        #    with open('./SPMfeaturelevel%d_train%d.pkl'%(i,i),'rb') as f:
        #        x_spm_train=pickle.load(f)
        print("Load SPM")
        
        #if not os.path.exists('./SPMfeaturelevel%d_test%d.pkl'%(i,i)):
        x_spm_test = [spatial_pyramid_matching(test_data[i],
                                               x_test_des[i],
                                               codebook,
                                               level=PYRAMID_LEVEL) for i in range(len(test_data))]
        x_spm_test = np.asarray(x_spm_test)
        '''
        
        if not os.path.exists('./SPMfeaturelevel%d_train%d.pkl'%(i,i)):
            x_spm_train = getHistogramSPM(i, train_data,codebook, 200)
            x_spm_test = getHistogramSPM(i, test_data,codebook, 200)
            with open('./SPMfeaturelevel%d_train%d.pkl'%(i,i),'wb') as f:
                pickle.dump(x_spm_train, f)
            with open('./SPMfeaturelevel%d_test%d.pkl'%(i,i),'wb') as f:
                pickle.dump(x_spm_test, f)
        
        else:
            with open('./SPMfeaturelevel%d_train%d.pkl'%(i,i),'rb') as f:
                x_spm_train=pickle.load(f)
            with open('./SPMfeaturelevel%d_test%d.pkl'%(i,i),'rb') as f:
                x_spm_test=pickle.load(f)
        #    with open('./SPMfeaturelevel%d_test%d.pkl'%(i,i),'wb') as f:
        #        pickle.dump(x_spm_test, f)
        #else:
        #    with open('./SPMfeaturelevel%d_test%d.pkl'%(i,i),'rb') as f:
        #        x_spm_test=pickle.load(f) 
        '''
        print("Load SPM")    
        
        
            
        #import pdb;pdb.set_trace()
        '''
        pca = PCA(n_components=100, svd_solver='randomized',whiten=True)
        
        x_spm_train=pca.fit_transform(x_spm_train)
        x_spm_test=pca.fit_transform(x_spm_test)'''
        print ("Done PCA")
        #SVM_Classify(x_spm_train,train_label,train_label,test_label,i)
        #predict=SVM_Classify(x_spm_train, train_label, x_spm_test, test_label,i)
        #import pdb;pdb.set_trace()
        #predict =svm_classifier(x_spm_train, train_label,x_spm_test )          ############Linear SVM
        
        predict=gramSVM(x_spm_train,x_spm_test,train_label)                      #########intersection
        
        predict=predict.reshape(-1,1)
        
        test_label=np.array(test_label)
        test_label=test_label.reshape(-1,1)
        #import pdb;pdb.set_trace()
        total_result=np.hstack([test_label,predict])
        
        df = pd.DataFrame(total_result,columns=["Id","Category"])
        df.to_csv("./gdrive/My Drive/result_dchan_level.csv",index=False,header=True)
        ! kaggle competitions submit -c 2019-ml-finalproject -f result_dchan_level.csv -m DCHAN_FINAL
        #subprocess.call(["kaggle","competitions","submit","-c", "2019-ml-finalproject", "-f","result_dchan_level.csv","-m", "DCHAN_FINAL"])
    